## Modeling Tutorial

In [1]:
### Import packages
import polars as pl
import pandas as pd
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
import itertools
from collections import Counter

# For plotting
import seaborn as sns

sns.set_theme(style="ticks")
sns.axes_style("darkgrid")
sns.set_theme()

from IPython.display import display, HTML

# For storing DataFrames
import pickle

import sys
from os import getcwd
sys.path.append(getcwd())
import MultiStagePackage.OptMSPfunctions as msp
from MultiStagePackage.models import *

from itertools import islice
import collections
from ast import literal_eval

# For optimization
from pygmo import *

### Model design

The following models below show how you can modify the provided models (the comments with 'ADD' show what was changed):

#### Case 1: More species in all models
- Second product (```P2```)
- Second substrate (```S2```)

In [2]:
### Prepare data

## Models
# Taken from models.py 

def model1(t, y):
    # Measurement-based parameters: 
    mu  = 0.13
    r_S = 16.13
    r_P = 26.11
    r_S2 = 15.00                # ADD new substrates
    r_P2 = 2.0                  # ADD new product 

    # Check for values smaller 0
    for i in range(len(y)):
        if(y[i]<0.0):
            y[i] = 0.0

    if(y[S]>0.001): 
        dXdt =   mu*y[X]
        dSdt = - r_S*y[X]
        dPdt =   r_P*y[X]
        dS2dt =  r_S2*y[X]       # ADD the second substrate to the ODE model
        dP2dt =  r_P2*y[X]       # ADD the second product to the ODE model
    
    # Case in which substrate was all taken up
    else:
        dXdt = 0
        dSdt = 0
        dPdt = 0
        dS2dt = 0               # ADD
        dP2dt = 0               # ADD

    if(dXdt<0.0):
        print(dXdt)
    
    dydt = [dXdt, dSdt, dPdt, dS2dt, dP2dt]     # ADD
    return(dydt)

def model2(t, y):
    # Measurement-based parameters:
    mu  = 0.06
    r_S = 22.86
    r_P = 41.25  
    r_S2 = 30.00                # ADD new substrates
    r_P2 = 10.0                 # ADD new product

    # Check for values smaller 0
    for i in range(len(y)):
        if(y[i]<0.0):
            y[i] = 0.0

    if(y[S]>0.001):
        dXdt =   mu*y[X]
        dSdt = - r_S*y[X]
        dPdt =   r_P*y[X]
        dS2dt = -r_S2*y[X]       # ADD the second substrate to the ODE model
        dP2dt =  r_P2*y[X]       # ADD the second product to the ODE model
    else:
        dXdt = 0
        dSdt = 0
        dPdt = 0
        dS2dt = 0               # ADD
        dP2dt = 0               # ADD
    if(dXdt<0.0):
        print(dXdt)
    
    dydt = [dXdt, dSdt, dPdt, dS2dt,dP2dt]      # ADD
    return(dydt)


models = [model1, model2]

## initial conditions
X_0 = 0.1    # gDW/L
S_0 = 100.0  # mmol/L 
P_0 = 0.0    # mmol/L

S2_0 = 500                      # ADD 
P2_0 = 700                      # ADD

s = np.array([X_0, S_0, P_0, S2_0, P2_0])       # ADD

## Indexing
X, S, P, S2, P2 = (i for i in range(len(s)))    # ADD

#### Case 2: More species but not in all models
- Second product (```P2```)
- Second substrate (```S2```)
- Oxygen **only** in the second model (```O_2```) 

In [18]:
### Prepare data

## Models
# Taken from models.py 

def model1(t, y):
    # Measurement-based parameters: 
    mu  = 0.13
    r_S = 16.13
    r_P = 26.11
    r_S2 = 15.00                # ADD new substrates reaction coefficient
    r_P2 = 2.0                  # ADD new product reaction coefficient
    r_O_2 = 0.0                 # ADD oxygen reaction coefficient

    # Check for values smaller 0
    for i in range(len(y)):
        if(y[i]<0.0):
            y[i] = 0.0

    if(y[S]>0.001): 
        dXdt =   mu*y[X]
        dSdt = - r_S*y[X]
        dPdt =   r_P*y[X]
        dS2dt = - r_S2*y[X]       # ADD the second substrate to the ODE model
        dP2dt =  r_P2*y[X]        # ADD the second product to the ODE model
        dO_2dt = r_O_2*y[X]       # ADD oxygen to the ODE model
    
    # Case in which substrate was all taken up
    else:
        dXdt = 0
        dSdt = 0
        dPdt = 0
        dS2dt = 0               # ADD 
        dP2dt = 0               # ADD 
        dO_2dt = 0             # ADD 

    if(dXdt<0.0):
        print(dXdt)
    
    dydt = [dXdt, dSdt, dPdt, dS2dt, dP2dt, dO_2dt]     # ADD
    return(dydt)

def model2(t, y):
    # Measurement-based parameters:
    mu  = 0.06
    r_S = 22.86
    r_P = 41.25  
    r_S2 = 30.00                # ADD new substrates reaction coefficient
    r_P2 = 10.0                 # ADD new product reaction coefficient
    r_O_2 = 2.0                 # ADD oxygen reaction coefficient 

    # Check for values smaller 0
    for i in range(len(y)):
        if(y[i]<0.0):
            y[i] = 0.0

    if(y[S]>0.001):
        dXdt =   mu*y[X]
        dSdt = - r_S*y[X]
        dPdt =   r_P*y[X]
        dS2dt = -r_S2*y[X]       # ADD the species in ODE model
        dP2dt =  r_P2*y[X]       # ADD the species in ODE model
        dO_2dt = r_O_2*y[X]      # ADD oxygen to the ODE model

    else:
        dXdt = 0
        dSdt = 0
        dPdt = 0
        dS2dt = 0               # ADD
        dP2dt = 0               # ADD
        dO_2dt = 0             # ADD
    if(dXdt<0.0):
        print(dXdt)
    
    dydt = [dXdt, dSdt, dPdt, dS2dt,dP2dt,dO_2dt]      # ADD
    return(dydt)


models = [model1, model2]

## initial conditions
X_0 = 0.1    # gDW/L
S_0 = 100.0  # mmol/L 
P_0 = 0.0    # mmol/L

S2_0 = 500                      # ADD 
P2_0 = 700                      # ADD
O_2_0 = 0.0                     # ADD

s = np.array([X_0, S_0, P_0, S2_0, P2_0, O_2_0])     # ADD

## Indexing
X, S, P, S2, P2, O_2 = (i for i in range(len(s)))    # ADD

### Test integration

In [19]:
%%time
times = [0, 1]  # Input time points
combi = [0, 1]   # Models
s = np.array([X_0, S_0, P_0, S2_0, P2_0, O_2_0])

# Test model1
result_num = sp.integrate.solve_ivp(models[int(combi[0])],t_span=[times[0], times[1]],y0=s,t_eval=[times[1]],dense_output=True,max_step=0.01,).y.T
print(result_num[0])

# Test model2
result_num = sp.integrate.solve_ivp(models[int(combi[1])],t_span=[times[0], times[1]],y0=s,t_eval=[times[1]],dense_output=True,max_step=0.01,).y.T
print(result_num[0])

[1.13882838e-01 9.82774601e+01 2.78831468e+00 4.98398134e+02
 7.00213582e+02 0.00000000e+00]
[1.06183655e-01 9.76440276e+01 4.25126257e+00 4.96908173e+02
 7.01030609e+02 2.06121822e-01]
CPU times: user 11.8 ms, sys: 3.86 ms, total: 15.6 ms
Wall time: 15 ms


### Test ```stage()``` function

In [20]:
print(msp.stage_num(combi=[0, 1], models=models, s=s, times=[0, 1, 24]))

[0.37182701630678777, 0.0007283279157452923, 180.1249370408921, 369.42604505139843, 743.2042784572463, 8.598139265810868, 20.721099641476254, 2]


### Test BruteForce

In [21]:
## Example: analytical-based 2-Stage with all time switch combinations possible between 0 and 24 with a minimal duration of 2 hour for each stage
t_start = 0
t_end = 24
min_duration = 5
n_stages = 2
combis = list(
    itertools.product([0, 1], [0, 1])
)  

MSP_2Stage = msp.doBruteForceNum(combis, models, n_stages, t_start, t_end, min_duration, s)
display(MSP_2Stage.to_pandas().sort_values(["Vol_P"], ascending=[False]).dropna(axis=0).drop_duplicates(subset="Models", keep="first")[:5])

,Index,Times,Models,End_T,End_X,End_S,End_P,finished,Vol_P,Y_SubInput,Y_SubUsed
59,60,"(0.0, 19.0, 24.0)","[2, 2]",368.77,0.36,0.0,180.44,743.744168744316,0.49,1.80,1.80
15,16,"(0.0, 5.0, 24.0)","[1, 2]",373.11,0.42,0.0,178.33,740.1834176100056,0.48,1.78,1.78
44,45,"(0.0, 19.0, 24.0)","[2, 1]",376.02,0.47,0.0,176.92,737.7981943623294,0.47,1.77,1.77
9,10,"(0.0, 14.0, 24.0)","[1, 1]",407.01,0.91,0.0,161.87,712.3991589531204,0.40,1.62,1.62


### Test Optimizer

In [22]:
def objective(self, x):
    res = msp.stage_num(combi=list(x[(self.max_stage-1):len(x)]), \
                                models=self.models, \
                                s=self.y, \
                                times=[self.tstart]+list(x[0:(self.max_stage-1)])+[self.tend], \
                                finished=0, \
                                step=self.step)
    # res = msp.stage_ana(combi=list(x[(self.max_stage-1):len(x)]), \
    #                             models=self.models, \
    #                             s=self.y, \
    #                             times=[self.tstart]+list(x[0:(self.max_stage-1)])+[self.tend], \
    #                             finished=0) 

    ## Objective
    # max. Vol. Productivity
    score = -res[P]/(res[3]-self.tstart)       
    # max. Titer
    #score = -res[P]
    # max. Yield 
    #score = -res[P]/(self.y[S]-res[S])


    ## Extra constraint (extracon)
    #con= res[P]/(self.y[S]-res[S])  # Yield
    #con2= res[P]                   # Titer
    #con3=res[S]                    # Substrate

    return score #,con    #,con2,con3

## 2.) Define the algorithm that will be used (in our case IHS) with the number of generations (fitness function evaluations) and a seed for reproducibility
# Note: per default there are always 10 evaluations so the total number of fitness function evaluations is 10 + the number of you pass to the gen attribute
algo_ihs = algorithm(
    ihs(gen=1000, seed=12345)
)  # total numer of evaluations = 10 + 1000
algo_ihs.set_verbosity(50)  # output best performing each 50th generation

## 3.) Define optimization problem
# Defining the problem through creating a msp.Optimizer object
problem_sim_volP = problem(
    decorator_problem(
        msp.Optimizer(
            s=s,
            models=models,
            tstart=0,
            tend=24,
            max_stage=2,
            min_duration=5,
            objective=objective
        ),
        fitness_decorator=msp.f_log_decor,
    )
)
# Define a relative tolerance that 
problem_sim_volP.c_tol = 1e-3

## 4.) Starting the optimization
res_opt = algo_ihs.evolve(
    population(problem_sim_volP, size=10, seed=12345)
)  # start with 10 candidate solutions


Fevals:          ppar:            bw:            dx:            df:      Violated:    Viol. Norm:        ideal1:
      1        0.35064       0.988553        2.24878      0.0916107              0              0      -0.489322
     51        0.38264       0.555904        7.47605    2.72332e-07              0              0      -0.489322
    101        0.41464       0.312608              0              0              0              0      -0.489322
    151        0.44664       0.175792              0              0              0              0      -0.489322
    201        0.47864      0.0988553              0              0              0              0      -0.489322
    251        0.51064      0.0555904              0              0              0              0      -0.489322
    301        0.54264      0.0312608       0.583981    2.57213e-07              0              0      -0.489322
    351        0.57464      0.0175792       0.583981    2.57213e-07              0             

In [24]:
log = pd.DataFrame(res_opt.problem.extract(decorator_problem).dv_log)
best_res_opt = msp.doOptDF(log, n_best=5)
best_res_opt_pd = msp.doConvert(best_res_opt, models, 0, 24, s)
best_res_opt_pd

,Index,Times,Models,End_T,End_X,End_S,End_P,finished,Vol_P,Y_SubInput,Y_SubUsed
0,1,"[0.0, 6.09, 24.0]","[1, 1]",368.77,0.36,0.0,180.45,743.7444233902177,0.49,1.8,1.8
1,2,"[0.0, 6.09, 24.0]","[1, 1]",368.77,0.36,0.0,180.45,743.7444233902177,0.49,1.8,1.8
2,3,"[0.0, 6.09, 24.0]","[1, 1]",368.77,0.36,0.0,180.45,743.7444233902177,0.49,1.8,1.8
3,4,"[0.0, 6.09, 24.0]","[1, 1]",368.77,0.36,0.0,180.45,743.7444233902177,0.49,1.8,1.8
4,5,"[0.0, 6.09, 24.0]","[1, 1]",368.77,0.36,0.0,180.45,743.7444233902177,0.49,1.8,1.8
